https://groups.google.com/forum/#!topic/recsys-challenge2020/piIOgvhQZFk

    We are observing strange patterns in the datasets and are wondering whether they are artificial or real. We are concerned about using features that may be artificial in our models. These features may increase local validation and increase public leaderboard but they may not predict the final test dataset.

    (Here is a hypothetical example. If all pseudo negatives are chosen from English language tweets, then there will be an unnatural pattern showing that English tweets have more negative targets. And if training data pseudo negatives are selected from English language tweets and test data pseudo negatives are chosen from Spanish language tweets, that will create a problem).


    I want to second this, I created some "dummy features" which makes NO SENSE AT ALL (think of features such as "Was it 35 celsius the day the tweet was made?"), and it got a huge improvement both on CV and Leaderboard. I think its clear something is not right, or I have no understanding how twitter works.


# start

In [114]:
trntmstmp=1584412344
valtmstmp=1585569704

import datetime
def showtm(tm): return datetime.datetime.fromtimestamp(tm).strftime('%Y-%m-%d %H:%M:%S')
print([showtm(tm) for tm in (trntmstmp, valtmstmp)])


nsmpl=int(1e6)

['2020-03-17 02:32:24', '2020-03-30 12:01:44']


# setup

In [115]:
grand_total=int(1.5e8)
valid_total=int(1.4e7)
print(f'trnsz {grand_total:,}, valsz {valid_total:,}')
tm_trn_min,tm_trn_max=(1580947200, 1581551999)
tm_val_min,tm_val_max=(1581552000, 1582156799)
print([showtm(tm) for tm in (tm_trn_min,tm_trn_max)])
print([showtm(tm) for tm in (tm_val_min,tm_val_max)])

trnsz 150,000,000, valsz 14,000,000
['2020-02-06 00:00:00', '2020-02-12 23:59:59']
['2020-02-13 00:00:00', '2020-02-19 23:59:59']


In [116]:
from tqdm import tqdm
from pathlib import Path
from collections import Counter
from sklearn.model_selection import train_test_split
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.metrics import accuracy_score, roc_auc_score, roc_curve, auc, precision_recall_curve
import pickle
import xgboost as xgb
import lightgbm as lgb

pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)


import datetime
def dtnow(): return datetime.datetime.now().strftime('%Y-%m-%d %H:%M:%S')

def iou(s1,s2):
    return len(s1&s2) / len(s1|s2)


SEED=101
HOME='/data/git/recsys20'
p_in=f'{HOME}/input'
# p_out=f'{HOME}/output/{PRFX}'
# Path(p_out).mkdir(exist_ok=True, parents=True)

from sklearn.metrics import precision_recall_curve, auc, log_loss

def compute_prauc(pred, gt):
    prec, recall, thresh = precision_recall_curve(gt, pred)
    prauc = auc(recall, prec)
    return prauc

def calculate_ctr(gt):
    positive = len([x for x in gt if x == 1])
    ctr = positive/float(len(gt))
    return ctr

def compute_rce(pred, gt):
    cross_entropy = log_loss(gt, pred)
    data_ctr = calculate_ctr(gt)
    strawman_cross_entropy = log_loss(gt, [data_ctr for _ in range(len(gt))])
    return (1.0 - cross_entropy/strawman_cross_entropy)*100.0

cols=['toks','hshtgs','twtid','media','links','domns','twttyp','lang','tm','u1id','u1_fllwer_cnt','u1_fllwing_cnt','u1_vrfed','u1_create_tm','u2id','u2_follower_cnt','u2_following_cnt','u2_vrfed','u2_create_tm','u1_fllw_u2','reply_tm','retwt_tm','retwt_cmmnt_tm','like_tm',]

cols_val = cols[:-4]
cols_tgt_tmstmp=[ 'retwt_tm', 'reply_tm', 'like_tm', 'retwt_cmmnt_tm',]
cols_tgt=[o.split('_tm')[0] for o in cols_tgt_tmstmp]
tgts             = ['Retweet','Reply','Like','RTwCmnt',]
assert cols_tgt == ['retwt',  'reply','like','retwt_cmmnt',]
ntgts=len(tgts)



# prep

In [117]:
%%time
dfval = pd.read_csv(f'{p_in}/val_{valtmstmp}.tsv',sep='\x01',header=None,names=cols_val,nrows=nsmpl)
lndfval=len(dfval)
print(f'{lndfval:,}')

1,000,000
CPU times: user 6.93 s, sys: 851 ms, total: 7.78 s
Wall time: 11 s


In [118]:
%%time
df = pd.read_csv(f'{p_in}/trn_{trntmstmp}.tsv',sep='\x01',header=None,names=cols,nrows=nsmpl)
lndf=len(df)
print(f'{lndf:,}')

1,000,000
CPU times: user 7.26 s, sys: 686 ms, total: 7.94 s
Wall time: 11.3 s


In [6]:
df.columns

Index(['toks', 'hshtgs', 'twtid', 'media', 'links', 'domns', 'twttyp', 'lang',
       'tm', 'u1id', 'u1_fllwer_cnt', 'u1_fllwing_cnt', 'u1_vrfed',
       'u1_create_tm', 'u2id', 'u2_follower_cnt', 'u2_following_cnt',
       'u2_vrfed', 'u2_create_tm', 'u1_fllw_u2', 'reply_tm', 'retwt_tm',
       'retwt_cmmnt_tm', 'like_tm'],
      dtype='object')

In [119]:
df[cols_tgt]=df[cols_tgt_tmstmp].notna()
df['enged']=df[cols_tgt].sum(1).astype(bool)

In [120]:
set_u1 = set(df.u1id)
set_u2 = set(df.u2id)

In [121]:
df['u1inu2']=df.u1id.isin(set_u2)
df['u2inu1']=df.u2id.isin(set_u1)

In [122]:
for col in cols_tgt+['enged']:
    print(col)
    display(df.groupby('u1inu2').enged.mean())
    display(df.groupby('u2inu1').enged.mean())
    

retwt


u1inu2
False    0.509397
True     0.589923
Name: enged, dtype: float64

u2inu1
False    0.506080
True     0.706563
Name: enged, dtype: float64

reply


u1inu2
False    0.509397
True     0.589923
Name: enged, dtype: float64

...

u1inu2
False    0.509397
True     0.589923
Name: enged, dtype: float64

u2inu1
False    0.506080
True     0.706563
Name: enged, dtype: float64

enged


u1inu2
False    0.509397
True     0.589923
Name: enged, dtype: float64

u2inu1
False    0.506080
True     0.706563
Name: enged, dtype: float64

In [28]:
dict(df.groupby('lang').size())

{'022EC308651FACB02794A8147AEE1B78': 2361,
 '0331BF70E606D62D92C96CE9AD71A7CF': 84,
 '06BEAB41D66CCFF329D1ED8BA120A6C2': 68,
 '06D61DCBBE938971E1EA0C38BD9B5446': 8887,
 '125C57F4FA6D4E110983FB11B52EFD4E': 2410,
 '167115458A0DBDFF7E9C0C53A83BAC9B': 3161,
 '190BA7DA361BC06BC1D7E824C378064D': 230,
 '1BC639981AE88E09129594B11F894A21': 8,
 '1FFD2FE4297F5E70EBC6C3230D95CB9C': 173,
 '2216D01F7B48554E4211021A46816FCF': 12,
 '22C448FF81263D4BAF2A176145EE9EAD': 16127,
 '259A6F6DFD672CB1F883CBEC01B99F2D': 18,
 '2996EB2FE8162C076D070A4C8D6532CD': 916,
 '3121F7240D488F74EEED9312E174B217': 21,
 '3820C29CBCA409A33BADF68852057C4A': 515,
 '3A85BCEC571C3F5AB1069E4924189177': 24,
 '3E16B11B7ADE3A22DDFC4423FBCEAD5D': 568,
 '4249CE88433AEA3F8DCEECF008B3CB95': 30,
 '4690215948DBF6872B8ED1C2BC87B17E': 1,
 '48236EC80FDDDFADE99420ABC9210DDF': 173,
 '4DC22C3F31C5C43721E6B5815A595ED6': 3288,
 '54208B51D44E7D91DC2F3DD02ADEDEC2': 20,
 '544FA32458C903F1125FE6598300A047': 30,
 '57ADD4576E2AD6648E9B2DE32F3462A5': 6,


In [29]:
dict(df.groupby('lang').like.mean())

{'022EC308651FACB02794A8147AEE1B78': 0.23252858958068615,
 '0331BF70E606D62D92C96CE9AD71A7CF': 0.5119047619047619,
 '06BEAB41D66CCFF329D1ED8BA120A6C2': 0.6470588235294118,
 '06D61DCBBE938971E1EA0C38BD9B5446': 0.3974344548216496,
 '125C57F4FA6D4E110983FB11B52EFD4E': 0.44481327800829873,
 '167115458A0DBDFF7E9C0C53A83BAC9B': 0.4194875039544448,
 '190BA7DA361BC06BC1D7E824C378064D': 0.44782608695652176,
 '1BC639981AE88E09129594B11F894A21': 0.875,
 '1FFD2FE4297F5E70EBC6C3230D95CB9C': 0.4277456647398844,
 '2216D01F7B48554E4211021A46816FCF': 0.75,
 '22C448FF81263D4BAF2A176145EE9EAD': 0.46859304272338315,
 '259A6F6DFD672CB1F883CBEC01B99F2D': 0.3333333333333333,
 '2996EB2FE8162C076D070A4C8D6532CD': 0.49672489082969434,
 '3121F7240D488F74EEED9312E174B217': 0.6190476190476191,
 '3820C29CBCA409A33BADF68852057C4A': 0.516504854368932,
 '3A85BCEC571C3F5AB1069E4924189177': 0.7083333333333334,
 '3E16B11B7ADE3A22DDFC4423FBCEAD5D': 0.6584507042253521,
 '4249CE88433AEA3F8DCEECF008B3CB95': 0.366666666666666